In [1]:
# Visualize MNIST dataset in Visdom
# Train MNIST
# Plot loss curve with Visdom

In [2]:
# First, open Terminal and type:

# python -m visdom.server

# Then, open browser and go to http://localhost:8097/

In [3]:
import torch

In [4]:
from torchvision.datasets import MNIST

In [5]:
from torchvision import transforms

In [6]:
# TODO: talk more about what these compose options do
transf = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, ), (1.0, ))])

In [7]:
train_set = MNIST(root='./data', train=True, transform=transf, download=False) # load training dataset

In [8]:
train_set.train_data.shape # see dataset size

torch.Size([60000, 28, 28])

In [9]:
train_set.train_data[0]

tensor([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,

In [10]:
# To visualize the samples in dataset, let's use Visdom
from visdom import Visdom
vis = Visdom()

In [11]:
vis.image(train_set.train_data[0]) # See image in Visdom page

'window_3629a842760a12'

In [12]:
train_set.train_labels[0]

tensor(5)

In [13]:
test_set = MNIST(root='./data', train=False, transform=transf, download=False) # load training dataset

In [14]:
from torchvision.utils import make_grid # Combine images into a nice-looking grid layout

In [15]:
# first 10 samples combined together
# MNIST dataset doesn't have channel dimension, so we use unsqueeze to manually add it
print(train_set.train_data.shape)
print(train_set.train_data.unsqueeze(1).shape)
combined_image = make_grid(train_set.train_data[:10].unsqueeze(1))

torch.Size([60000, 28, 28])
torch.Size([60000, 1, 28, 28])


In [16]:
vis.image(combined_image)

'window_3629a8428cca40'

In [17]:
# Better way to visualize how good our model does
# For each target value (0-9), see how often our model gets it right
def confusion(model, n, dataset):
    conf = torch.zeros(n, n)
    model.eval() # Put model in evaluation mode
    for data, target in dataset:
        data = data.unsqueeze(1)
        output = model(data)
        _, pred = torch.max(output.data, 1)
        conf[target][pred[0]] += 1
        
    # Normalize
    for i in range(n):
        conf[i] = conf[i] / conf[i].sum()
        
    vis.image(conf)
    
# A perfect model should show a diagonal white line

In [18]:
import torch.nn as nn
import torch.nn.functional as F

In [19]:
# Define our own CNN
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.fc1 = nn.Linear(20 * 4 * 4, 50)
        self.fc2 = nn.Linear(50, 10)
        
    def forward(self, x):
        # we can also use functional interface for Conv2d and pass in the weights and biases, useful in weight sharing scenario
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 20 * 4 * 4) # -1 means we only care about the size of the last dimension, and just merge the rest of the dimensions into one
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=-1)

In [20]:
# Before we train the model, let's define the method for plotting the loss curve
import numpy as np
def plot_loss_curve(epoch, nepochs, loss, accuracy):
    vis.line(
        X = np.column_stack(([epoch], [epoch])),
        Y = np.column_stack(([loss], [accuracy])),
        win = 'loss_curve',
        opts = dict(
            title='MNIST loss curve', # Set the title of the plot
            legend=['Loss', 'Accuracy'], # Set name for each curve
            xtickmin=0,
            xtickmax=nepochs,
            xtickstep=0.01,
            ytickmin=0,
            ytickmax=1.5,
            ytickstep=0.01
        ),
        update = None if epoch == 0 else 'append'
    )

In [21]:
from torch import optim

In [22]:
from torch.optim import lr_scheduler

In [23]:
from torch.utils.data import TensorDataset, DataLoader

In [24]:
def train_model(model, train_set, test_set, nepochs):
    batch_size = 500
    # dataloader can help you get batched data and shuffle the data
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_set)
    
    optimizer = optim.SGD(model.parameters(), lr=0.03, momentum=0.5)
    
    model.train()
    
    for epoch in range(nepochs):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            optimizer.zero_grad() # This will zero out all the gradients for the model's parameters
            output = model(data) # This is a softmax output over the possible labels
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()
            
            if batch_idx % 50 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), float(loss)))

        model.eval()
        test_loss = 0.0
        correct = 0
        for data, target in test_loader:
            output = model(data)
            test_loss += F.nll_loss(output, target, size_average=False).data.item()
            
            _, pred = torch.max(output.data, 1)
            
            correct += (pred == target.data).sum()
            
        test_loss /= len(test_set)
        accuracy = float(correct)/len(test_set)
        print("Loss: {:.6f}".format(test_loss), "Accuracy: ", accuracy)
        plot_loss_curve(epoch, nepochs, test_loss, accuracy)

In [25]:
model = CNN()

In [26]:
train_model(model, train_set, test_set, 10)

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.301107
Train Epoch: 0 [25000/60000 (42%)]	Loss: 2.261651
Train Epoch: 0 [50000/60000 (83%)]	Loss: 1.925754
Loss:  1.1276718949869275 Accuracy:  0.7508
Train Epoch: 1 [0/60000 (0%)]	Loss: 1.165561
Train Epoch: 1 [25000/60000 (42%)]	Loss: 0.419964
Train Epoch: 1 [50000/60000 (83%)]	Loss: 0.420000
Loss:  0.34246425257586816 Accuracy:  0.8935
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.394286
Train Epoch: 2 [25000/60000 (42%)]	Loss: 0.316114
Train Epoch: 2 [50000/60000 (83%)]	Loss: 0.224696
Loss:  0.21831342190593422 Accuracy:  0.9312
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.278661
Train Epoch: 3 [25000/60000 (42%)]	Loss: 0.230181
Train Epoch: 3 [50000/60000 (83%)]	Loss: 0.195008
Loss:  0.163646366741533 Accuracy:  0.95
Train Epoch: 4 [0/60000 (0%)]	Loss: 0.154957
Train Epoch: 4 [25000/60000 (42%)]	Loss: 0.170631
Train Epoch: 4 [50000/60000 (83%)]	Loss: 0.167735
Loss:  0.12565901902516813 Accuracy:  0.9603
Train Epoch: 5 [0/60000 (0%)]	Loss: 0.104393
Train Epoc

In [27]:
confusion(model, 10, test_set) # See confusion matrix

In [28]:
# Save model state as checkpoint, to be used later
torch.save(model.state_dict(), 'mnist.pth')

# Load model into another variable
new_model = CNN()
new_model.load_state_dict(torch.load('mnist.pth'))

In [29]:
new_model

CNN(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [30]:
new_model

CNN(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [31]:
model

CNN(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [32]:
# TODO: for transfer learning, make sure the two linear layers are replaced